In [1]:
import pandas as pd
import os 

In [21]:
ruta_archivos = "D:/PROYECTO FINAL/Google Maps/metadata-sitios/"     # Ruta de los json de Sitios
archivos_json = [[os.path.join(ruta_archivos, archivo), archivo] for archivo in os.listdir(ruta_archivos) if archivo.endswith('.json')]

In [3]:
archivos_json

[['D:/PROYECTO FINAL/Google Maps/metadata-sitios/1.json', '1.json'],
 ['D:/PROYECTO FINAL/Google Maps/metadata-sitios/2.json', '2.json']]

In [79]:
#df_estados_USA= pd.read_csv("D:/PROYECTO FINAL/Google Maps/estados_usa.csv",delimiter = ';', encoding = "utf-8")
df_estados_USA= pd.read_csv("C:/Data/GMaps/estados_usa.csv",delimiter = ';', encoding = "utf-8")

In [80]:
#df_sitios= pd.read_json('D:/PROYECTO FINAL/Google Maps/metadata-sitios/1.json', lines=True)
df_sitios= pd.read_json('C:/Data/GMaps/sitios/1.json', lines=True)

In [81]:
df_sitios.shape

(275001, 15)

In [82]:
df_sitios.drop_duplicates('gmap_id',inplace=True)

In [83]:
df_sitios.shape

(248428, 15)

In [84]:
df_sitios= df_sitios.dropna(subset=['address']) # Elimina filas con address nulas
df_sitios= df_sitios.dropna(subset=['category']) # Elimina filas con category nulas

In [87]:
df_sitios.shape

(237332, 15)

In [88]:
df_sitios['Es_Restaurant'] = df_sitios['category'].apply(lambda x: 'Restaurant' in x)

In [89]:
df_sitios= df_sitios[df_sitios['Es_Restaurant']]

In [90]:
df_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4867 entries, 6 to 274962
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              4867 non-null   object 
 1   address           4867 non-null   object 
 2   gmap_id           4867 non-null   object 
 3   description       1089 non-null   object 
 4   latitude          4867 non-null   float64
 5   longitude         4867 non-null   float64
 6   category          4867 non-null   object 
 7   avg_rating        4867 non-null   float64
 8   num_of_reviews    4867 non-null   int64  
 9   price             1329 non-null   object 
 10  hours             3909 non-null   object 
 11  MISC              4837 non-null   object 
 12  state             3947 non-null   object 
 13  relative_results  3107 non-null   object 
 14  url               4867 non-null   object 
 15  Es_Restaurant     4867 non-null   bool   
dtypes: bool(1), float64(3), int64(1), object(11)


In [91]:
df_sitios= df_sitios.fillna({'price':'SIN DATO', 'state':'SIN DATO'})     # Se imputan los valores nulos a 'SIN DATO'

In [92]:
# En pruebas se detecto que esta fila estaba dando problemas con la funcion para extraer, asi que se procede a eliminarla
df_sitios= df_sitios[df_sitios['address'] != "〒10028 New York, Lexington Ave, (New) Ichie Japanese Restaurant"]

In [93]:
def Ext_Ciudad_Estado(dir):
    ciudad= "SIN DATO"
    estado= "SIN DATO"     
    if len(str(dir)) > 10:  
        lista= str(dir).split(',')
        if len(lista) > 2:
            codigo= lista[-1][1:3]
            df_filtro= df_estados_USA[df_estados_USA['nombre_corto'].str.contains(codigo)]        
            if not df_filtro.empty:            
                ciudad = lista[-2].strip()
                estado= df_filtro.nombre_largo.values[0].strip()
    return ciudad, estado

In [94]:
df_sitios[['city','state']] = df_sitios.apply(lambda x: Ext_Ciudad_Estado(x['address']), axis=1, result_type='expand')

In [95]:
df_sitios.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,Es_Restaurant,city
6,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,None,33.916402,-118.010855,[Restaurant],4.5,18,SIN DATO,"[[Thursday, 11AM–9:30PM], [Friday, 11AM–9:30PM...","{'Service options': ['Outdoor seating', 'Curbs...",California,None,https://www.google.com/maps/place//data=!4m2!3...,True,La Mirada
68,Vivi Bubble Tea,"Vivi Bubble Tea, 701 S 5th St, Philadelphia, P...",0x89c6c89efcaed69d:0xded973f6033e7dba,"Bubble tea shop offering hot & cold versions, ...",39.940293,-75.150923,[Restaurant],4.0,8,SIN DATO,None,"{'Service options': ['Delivery'], 'Amenities':...",Pennsylvania,"[0x89c6c62a40f4b733:0x73bfeec1898f272b, 0x89c6...",https://www.google.com/maps/place//data=!4m2!3...,True,Philadelphia
96,Hale Pops,"Hale Pops, 55-370 Kamehameha Hwy, Laie, HI 96762",0x7c00456eecad3111:0x8217f9600c51f33,None,21.637796,-157.920714,[Restaurant],4.4,18,SIN DATO,"[[Thursday, 11AM–8PM], [Friday, 11AM–8PM], [Sa...","{'Service options': ['Outdoor seating', 'Takeo...",Hawái,"[0x7c00451360f80cf1:0x930291a38bab3132, 0x7c00...",https://www.google.com/maps/place//data=!4m2!3...,True,Laie


In [96]:
lista_estados= ['Florida', 'Pennsylvania', 'Tennessee', 'California', 'Texas', 'New York']
df_sitios= df_sitios[df_sitios['state'].isin(lista_estados)]

In [97]:
df_sitios.shape

(1998, 17)

In [98]:
df_category= df_sitios[['gmap_id','category']]
df_category= df_category.explode('category')

In [99]:
df_category['category'].value_counts()

category
Restaurant                 1998
Sandwich shop               150
Fast food restaurant        148
Takeout Restaurant          137
Caterer                     128
                           ... 
Haitian restaurant            1
Bed & breakfast               1
Motel                         1
Performing arts theater       1
Cantonese restaurant          1
Name: count, Length: 272, dtype: int64

In [100]:
df_category= df_category.rename(columns={'gmap_id': 'business_id','category': 'category_name'}) # cambiar nombre de las columnas

In [102]:
df_category.head(3)

,business_id,category_name
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Restaurant
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Restaurant
114,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Restaurant


In [103]:
# Se guardan las columnas MISC y el gmap_id en un df
df_misc= df_sitios[['gmap_id','MISC']]
df_misc= df_misc.dropna(subset=['MISC']) # Elimina filas con MISC nulo

In [104]:
# Funcion para cargar los json a una serie de pandas
def expandir_diccionario(diccionario):
    return pd.Series(diccionario)

# Se concatenas las series al df original
df_expandido = pd.concat([df_misc, df_misc['MISC'].apply(expandir_diccionario)], axis=1)
df_expandido.drop('MISC', axis=1, inplace=True)     # Se elimina la columna de MISC
df_misc= df_expandido

In [105]:
df_Service_options= df_misc[['gmap_id','Service options']]      # Se crea un df solo las columnas gmap_id y Service Options
df_Service_options= df_Service_options.rename(columns={'gmap_id': 'business_id','Service options': 'service_option'}) # cambiar nombre de las columnas
df_Service_options= df_Service_options.dropna(subset=['service_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Service_options= df_Service_options.explode('service_option')

In [106]:
df_Service_options.head(3)

,business_id,service_option
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Outdoor seating
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Curbside pickup
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,No-contact delivery


In [107]:
df_Planning= df_misc[['gmap_id','Planning']]      # Se crea un df solo las columnas gmap_id y Planning
df_Planning= df_Planning.rename(columns={'gmap_id': 'business_id', 'Planning': 'planning_option'}) # cambiar nombre de la columna
df_Planning= df_Planning.dropna(subset=['planning_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Planning= df_Planning.explode('planning_option')

In [108]:
df_Planning.head(3)

,business_id,planning_option
917,0x89c587762e7e69dd:0x30a60dce48b073c3,Accepts reservations
1165,0x88c2fea614c52c8b:0x50e166caa05e8c0f,Accepts reservations
2147,0x89c25a275e5088dd:0xb6ecb4eaa82d1180,Quick visit


In [109]:
#Borrar Columnas
df_sitios= df_sitios.drop(['relative_results','address', 'num_of_reviews', 'description', 'url','category', 'MISC', 'hours'], axis=1) 
# Ordena el orden de las columnas
df_sitios= df_sitios[['gmap_id','name', 'city', 'state', 'latitude', 'longitude', 'avg_rating', 'price']]

In [110]:
df_sitios= df_sitios.rename(columns={'gmap_id': 'business_id', 'avg_rating': 'stars'}) # cambiar nombre de la columna

In [111]:
df_sitios['platform']= 1

In [112]:
df_sitios

,business_id,name,city,state,latitude,longitude,stars,price,platform
6,0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,Vons Chicken,La Mirada,California,33.916402,-118.010855,4.5,SIN DATO,1
68,0x89c6c89efcaed69d:0xded973f6033e7dba,Vivi Bubble Tea,Philadelphia,Pennsylvania,39.940293,-75.150923,4.0,SIN DATO,1
114,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,Cape Coral,Florida,26.641377,-81.940545,5.0,$$,1
123,0x8644b59b8fe872e5:0x5e638876caa84cc3,Golden Castle,Austin,Texas,30.273985,-97.719563,4.5,SIN DATO,1
271,0x808f87f90c1f661f:0xf384e804a61e0c0b,Mariscos el poblano,Oakland,California,37.764203,-122.214647,5.0,SIN DATO,1
...,...,...,...,...,...,...,...,...,...
274722,0x89e82d0591b06633:0x40e962ba688eb743,Deli boss & brunch,West Babylon,New York,40.719946,-73.355972,4.4,SIN DATO,1
274772,0x887f850c749005f9:0xed634f0483c645a4,Taqueria Los Compadres,Memphis,Tennessee,35.050547,-89.866261,4.9,SIN DATO,1
274781,0x89c2f404caa68913:0x592c8b89b6226a82,Pica Pollo 809,New York,New York,40.855985,-73.928615,4.1,SIN DATO,1
274958,0x89d6b577c1295421:0xb1f4ce1abf53d3b6,Palermo’s Market,Rochester,New York,43.157746,-77.605956,4.9,SIN DATO,1
